In [32]:
import pandas as pd
import math

In [33]:
pickup_CauGiay = ["07:00:00", "09:30:00", "10:25:00", "11:40:00", "13:30:00"]
pickup_HaDong = ["07:30:00", "09:30:00", "10:30:00", "11:30:00", "13:30:00"]
pickup_HaiBaTrung = ["07:40:00", "09:40:00", "10:40:00", "12:00:00", "13:30:00"]
pickup_TayHo = ["07:40:00", "09:40:00", "10:40:00", "11:40:00", "13:30:00"]

pickup_time = pickup_CauGiay + pickup_HaDong + pickup_HaiBaTrung + pickup_TayHo
pickup_time = list(set(pickup_time))

dct_pickup_time = {
    "Cau Giay": pickup_CauGiay,
    "Hai Ba Trung": pickup_HaiBaTrung,
    "Ha Dong": pickup_HaDong,
    "Tay Ho": pickup_TayHo
}

In [34]:
# bus_schedule_file = "../data/bus.xlsx"
bus_schedule_file = "../data/Bus_schedules_for_Chatbot_20240816.xlsx"

In [35]:
filepath = "../data/StartingTime.csv"
df_bus = pd.read_csv(filepath)
df_bus

,route_name,pickup_point,dropoff_point,date,slot1,slot2,slot3,slot4,slot5,slot6
0,Cau Giay,Nguyen Quoc Tri,National Cinema Centre,NaN,07:20,09:35,10:25,11:40,13:30,NaN
1,Cau Giay,Nguyen Quoc Tri,BUV Campus,NaN,07:20,09:35,10:25,11:40,13:30,NaN
2,Cau Giay,National Cinema Centre,BUV Campus,NaN,07:35,09:45,10:40,11:55,13:45,NaN
3,Cau Giay,BUV Campus,National Cinema Centre,NaN,11:30,12:30,13:30,15:15,17:35,18:15
4,Cau Giay,BUV Campus,Nguyen Quoc Tri,NaN,11:30,12:30,13:30,15:15,17:35,18:15
...,...,...,...,...,...,...,...,...,...,...
133,Ecopark,Westbay – B Building,Aquabay - Sky 2 Building,NaN,11:36,12:36,13:36,15:21,17:41,18:21
134,Ecopark,Westbay – B Building,Aquabay – Park 1 - Building,NaN,11:36,12:36,13:36,15:21,17:41,18:21
135,Ecopark,Westbay – C Building,Aquabay - Sky 2 Building,NaN,11:36,12:36,13:36,15:21,17:41,18:21
136,Ecopark,Westbay – C Building,Aquabay – Park 1 - Building,NaN,11:36,12:36,13:36,15:21,17:41,18:21


In [36]:
# Get the list of sheet names
sheet_names = pd.ExcelFile(bus_schedule_file).sheet_names
sheet_names

['Cau Giay',
 'Ha Dong',
 'Hai Ba Trung',
 'Ecopark',
 'Tay Ho',
 '15.07 - 19.07',
 '22.07 - 26.07',
 '29.07 - 02.08',
 '05.07 - 09.08',
 '12.08 - 17.08',
 '19.08 - 24.08']

In [37]:
# Pick the latest sheet
latest_sheet = sheet_names[-1]

In [38]:
# Load the data from the latest sheet
df = pd.read_excel(bus_schedule_file, sheet_name=latest_sheet)
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BẢNG TỔNG HỢP SỐ LƯỢNG XE TRONG TUẦN | WEEKLY ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
df_calendar = df.iloc[8:, :]
df_calendar

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
8,NaN,NaN,NaN,1900-01-01 07:19:30,1900-01-01 09:29:30,10:30,11:30,13:30,11:30,12:30,1900-01-01 13:29:30,15:15,17:35,1900-01-01 18:44:30
9,2024-08-19 00:00:00,Mon,HBT,1,1,1,1,0,1,1,0,1,1,1
10,2024-08-19 00:00:00,Mon,CG,1,1,1,1,0,1,1,0,1,1,1
11,2024-08-19 00:00:00,Mon,TH,1,0,1,1,0,1,1,0,1,1,1
12,2024-08-19 00:00:00,Mon,HD,1,0,1,1,0,1,1,0,1,1,1
13,2024-08-20 00:00:00,Tue,HBT,1,1,0,0,1,1,1,1,1,1,1
14,2024-08-20 00:00:00,Tue,CG,1,1,0,0,1,1,1,1,1,1,1
15,2024-08-20 00:00:00,Tue,TH,1,1,0,0,1,1,1,0,1,1,1
16,2024-08-20 00:00:00,Tue,HD,1,1,0,0,1,1,1,0,1,1,1
17,2024-08-21 00:00:00,Wed,HBT,1,1,1,0,0,0,1,1,1,1,1


In [40]:
state = {
    'route name': ['Tay Ho'], 
    'pick-up point': ['Ho Tay Water Park'], 
    'drop-off point': ['BUV Campus'], 
    'specific date': ['Tue 20 Aug'], 
    'specific time': []
}

In [41]:
dct_excel_mapping = {
    "Hai Ba Trung": "HBT",
    "Tay Ho": "TH",
    "Cau Giay": "CG",
    "Ha Dong": "HD"
}

In [42]:
route_name = state["route name"][0]
pickup_point = state["drop-off point"][0] # ERROR
dropoff_point = state["pick-up point"][0] # ERROR
specific_date = state["specific date"][0]

In [43]:
def is_empty_or_none(lst):
    # print("type:", type(lst))
    if (type(lst)==list) and (None in lst):
        lst = [item for item in lst if item is not None]
    return lst is None or len(lst) == 0

In [44]:
def gen_available_timeslot(result_calendar, result):
    suggestion = []
    for col in result_calendar.columns:
        if (result_calendar[col].values[0]==1) and (int(col)<=7) and (pickup_point!="BUV Campus"):
            # print(col, result_calendar[col].values[0])
            timeslot = result["slot"+str(int(col)-2)].values[0]
            suggestion.append(str(timeslot))
        if (result_calendar[col].values[0]==1) and (int(col)>=8) and (pickup_point=="BUV Campus"):
            timeslot = result["slot"+str(int(col)-7)].values[0]
            suggestion.append(str(timeslot))
    suggestion = [x for x in suggestion if x!="nan"]     
    return suggestion

In [45]:
def generate_specific_time(state):
    route_name = state["route name"][0]
    pickup_point = state["pick-up point"][0]
    dropoff_point = state["drop-off point"][0]
    specific_date = state["specific date"][0]

    # if there is a route name
    if is_empty_or_none(state["route name"])==False:

        # if no pick-up point:
        if is_empty_or_none(state["pick-up point"])==True:
            # then return all starting time available of this route
            return dct_pickup_time[state["route name"]]

        # if there is a pick-up point
        if is_empty_or_none(state["pick-up point"])==False:
            # then return starting time of this pickup point only
            print("---------get suggest time here--------------")
            weekday = specific_date.split()[0]
            result_calendar = df_calendar[(df_calendar["Unnamed: 1"]==weekday)
                            & (df_calendar["Unnamed: 2"]==dct_excel_mapping[route_name])]
            result_calendar.columns = [str(i) for i in range(0, 14)]
            result = df_bus[(df_bus["route_name"]==route_name) \
                & (df_bus["pickup_point"] == pickup_point)
                & (df_bus["dropoff_point"] == dropoff_point)]

            suggestion = gen_available_timeslot(result_calendar, result)
            print("suggestion:", suggestion)
            return suggestion

    # if no route name
    if is_empty_or_none(state["route name"])==True:

        # if no pick-up point:
        if is_empty_or_none(state["pick-up point"]) == True:
            return pickup_time

        # if there is a pick-up point
        if is_empty_or_none(state["pick-up point"]) == False:
            suggestion = gen_available_timeslot()
            
            return suggestion

In [46]:
generate_specific_time(state)

---------get suggest time here--------------
suggestion: ['07:40', '09:40', '11:40', '13:30']


['07:40', '09:40', '11:40', '13:30']

In [31]:
pickup_point

'BUV Campus'